In [1]:
input_data = {
    "Amsterdam": {
        "Arrival Date": "2025-06-01",
        "Departure Date": "2025-06-03",
        "Airport": "AMS"
    },
    "The Hague": {
        "Arrival Date": "2025-06-03",
        "Departure Date": "2025-06-05",
        "Airport": "RTM"                      
    },
    "Maastricht": {
        "Arrival Date": "2025-06-05",
        "Departure Date": "2025-06-08",
        "Airport": "MST"
    },
    "Eindhoven": {
        "Arrival Date": "2025-06-08",
        "Departure Date": "2025-06-10",
        "Airport": "EIN"
    }
}

In [2]:
import http.client
import urllib.parse

def JSONconverter(input):
    cities = list(input.keys())
    arrivals = []
    departures = []
    airports = []

    encoded_cities = []  # To store URL-encoded city names

    for keys, values in input.items():
        encoded_city = urllib.parse.quote(keys)  # URL-encode the city name
        encoded_cities.append(encoded_city)
        
        arrivals.append(values["Arrival Date"])
        departures.append(values["Departure Date"])
        airports.append(values["Airport"])

    return cities,encoded_cities, arrivals, departures, airports


In [3]:
# def restructure_hotel_data(raw_data):
#     structured_data = []

#     for city, rankings in raw_data.items():
#         city_entry = {
#             "city": city,
#             "hotels_by": {}
#         }
#         for criterion, hotels in rankings.items():
#             city_entry["hotels_by"][criterion] = [
#                 {"name": hotel_name, "price": round(price, 2)} for hotel_name, price in hotels
#             ]
#         structured_data.append(city_entry)

#     return structured_data

def restructure_hotel_data(hotel_data):
    restructured = {}

    for city, sorting_data in hotel_data.items():
        restructured[city] = {}
        for sort_type, hotels in sorting_data.items():
            restructured[city][sort_type] = [
                {
                    "name": name,
                    "price": price,
                    "review_score": review_score
                }
                for name, price, review_score in hotels
            ]

    return restructured


In [4]:
import json
import http.client
def HotelRetriever(input,numadults):
    realcities,cities,arrivals,departures,_ = JSONconverter(input)
    conn = http.client.HTTPSConnection("booking-com15.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': "cd3649e95amsh19e77450d847972p129963jsn55e782c26fc6",
        'x-rapidapi-host': "booking-com15.p.rapidapi.com"
    }
    FinalHotelArray = {}
    for i in range(len(cities)):
        real_city_name = realcities[i]
        city_name = cities[i]
        arrival_date=arrivals[i]
        departure_date=departures[i]
        conn.request("GET", f"/api/v1/hotels/searchDestination?query={city_name}", headers=headers)
        res = conn.getresponse()
        data = res.read()
        data_json = json.loads(data.decode("utf-8"))
        for dest in data_json['data']:
            if dest['search_type']=="city":
                dest_id = dest['dest_id']
                break
        Sortings = ['bayesian_review_score','popularity','price']
        HotelArray = {}
        for sorts in Sortings:
            url = f"/api/v1/hotels/searchHotels?dest_id={dest_id}&search_type=CITY&adults={numadults}&arrival_date={arrival_date}&departure_date={departure_date}&sort_by={sorts}&currency_code=USD"
            conn.request("GET", url, headers=headers)
            res = conn.getresponse()
            data = res.read()
            data_json = json.loads(data.decode("utf-8"))
            num=0
            for hotels in data_json['data']['hotels']:
                if num==3:
                    break
                if sorts not in HotelArray:
                    HotelArray[sorts] = [(hotels['property']['name'], hotels['property']['priceBreakdown']['grossPrice']['value'],hotels['property']['reviewScore'])]
                else:
                    HotelArray[sorts].append((hotels['property']['name'], hotels['property']['priceBreakdown']['grossPrice']['value'],hotels['property']['reviewScore']))
                num+=1
            FinalHotelArray[real_city_name]=HotelArray
    return restructure_hotel_data(FinalHotelArray)

In [5]:
HotelRetriever(input_data,3)

{'Amsterdam': {'bayesian_review_score': [{'name': 'B&B Zunderdorp',
    'price': 466.177529554048,
    'review_score': 9.5},
   {'name': 'Hotel TwentySeven - Small Luxury Hotels of the World',
    'price': 3605.86639964992,
    'review_score': 9.8},
   {'name': 'B&B Modern Times',
    'price': 832.439419501884,
    'review_score': 9.6}],
  'popularity': [{'name': 'Inntel Hotels Amsterdam Landmark',
    'price': 479.770013595527,
    'review_score': 8.2},
   {'name': 'Via Amsterdam', 'price': 320.492717031151, 'review_score': 8},
   {'name': 'Joy Hotel', 'price': 372.64415111891, 'review_score': 7.6}],
  'price': [{'name': 'Hostelle - women only hostel Amsterdam',
    'price': 171.563327646502,
    'review_score': 8.6},
   {'name': 'Princess Hostel Leidse Square Amsterdam',
    'price': 194.716785581096,
    'review_score': 4},
   {'name': 'Amsterdam Hostel Leidseplein',
    'price': 205.116863407455,
    'review_score': 6.3}]},
 'The Hague': {'bayesian_review_score': [{'name': 'The Col

In [6]:
import http
import json
conn = http.client.HTTPSConnection("booking-com15.p.rapidapi.com")
headers = {
        'x-rapidapi-key': "cd3649e95amsh19e77450d847972p129963jsn55e782c26fc6",
        'x-rapidapi-host': "booking-com15.p.rapidapi.com"
    }
conn.request("GET", f"/api/v1/hotels/searchDestination?query=austin", headers=headers)
res = conn.getresponse()
data = res.read()
data_json = json.loads(data.decode("utf-8"))


In [7]:
for dest in data_json['data']:
    if dest['search_type']=="city":
        dest_id = dest['dest_id']
        break

url = f"/api/v1/hotels/searchHotels?dest_id={dest_id}&search_type=CITY&adults=5&arrival_date=2025-06-05&departure_date=2025-06-08&currency_code=USD"
conn.request("GET", url, headers=headers)
res = conn.getresponse()
data = res.read()
data_json = json.loads(data.decode("utf-8"))

In [8]:
data_json

{'status': True,
 'message': 'Success',
 'timestamp': 1745177188424,
 'data': {'hotels': [{'hotel_id': 491907,
    'accessibilityLabel': 'Comfort Suites.\n3 out of 5 stars.\n9.0 Superb 283 reviews.\n\u200e42 km from centre\u202c.\n3\xa0beds.\n547 USD.\nIncludes taxes and charges.\nFree cancellation.\nNo prepayment needed.',
    'property': {'position': 0,
     'reviewScoreWord': 'Superb',
     'mainPhotoId': 334397689,
     'checkinDate': '2025-06-05',
     'isPreferred': True,
     'photoUrls': ['https://cf.bstatic.com/xdata/images/hotel/square500/334397689.jpg?k=bd44545fe0ef2ecac857f4c08a7ce586eb1bf668f0af3c19c0d5a4959ef7be0a&o=',
      'https://cf.bstatic.com/xdata/images/hotel/square1024/334397689.jpg?k=bd44545fe0ef2ecac857f4c08a7ce586eb1bf668f0af3c19c0d5a4959ef7be0a&o=',
      'https://cf.bstatic.com/xdata/images/hotel/square2000/334397689.jpg?k=bd44545fe0ef2ecac857f4c08a7ce586eb1bf668f0af3c19c0d5a4959ef7be0a&o='],
     'checkout': {'untilTime': '12:00', 'fromTime': '00:00'},
    